# Zadanie 9

In [3]:
import numpy as np
from math import ceil
import itertools

In [4]:
def majority(neighbours):
    return int(sum(neighbours) > 1)


def GKL(initial_config, j, k, steps):
    diagram = [initial_config]
    n = len(initial_config)
    padded_config = np.pad(initial_config, (k, k), 'wrap')

    for _ in range(steps):
        neighbourhoods = []
        for idx in range(k, n + k):
            if padded_config[idx] == 0:
                neighbourhoods.append([padded_config[idx], padded_config[idx - j], padded_config[idx - k]])
            else:
                neighbourhoods.append([padded_config[idx], padded_config[idx + j], padded_config[idx + k]])

        result = [majority(neighbourhood) for neighbourhood in neighbourhoods]
        diagram.append(result)
        padded_config = np.pad(result, (k, k), 'wrap')

    return diagram


def DCP(diagram):
    initial_density = np.mean(diagram[0])  # Calculate initial density
    final_density = np.mean(diagram[-1])  # Calculate final density

    if initial_density < 0.5:
        return final_density == 0
    return final_density == 1


In [5]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)

            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)

In [73]:
GKL([0, 1, 1, 0, 0, 1, 0, 0], 1, 3, 3)

[[0, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [75]:
DCP(GKL([0, 1, 1, 0, 0, 1, 0, 0], 1, 3, 2))

False

In [4]:
configurations = configuration_reader("ALL_N21.bin", 21, True)

In [5]:
rules = [(4, 12), (3, 9), (2, 6), (5, 15), (1, 3), (1, 9), (1, 11), (2, 14), (2, 10), (1, 7), (1, 5)]

for rule in rules:
    results = []
    j = rule[0]
    k = rule[1]

    for config in configurations:
        results.append(DCP(GKL(config, j, k, 42)))

    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')


Rule (4, 12): 90.78% of correct solutions
Rule (3, 9): 17.67% of correct solutions
Rule (2, 6): 90.78% of correct solutions
Rule (5, 15): 90.78% of correct solutions
Rule (1, 3): 90.78% of correct solutions
Rule (1, 9): 67.6% of correct solutions
Rule (1, 11): 12.85% of correct solutions
Rule (2, 14): 36.01% of correct solutions
Rule (2, 10): 75.22% of correct solutions
Rule (1, 7): 36.01% of correct solutions
Rule (1, 5): 75.22% of correct solutions


# Zadanie 11

In [4]:
configurations = configuration_reader("ALL_N23.bin", 23, True)

In [6]:
def binary_array_to_int(array):
    binary_string = ''.join(str(bit) for bit in array)
    return int(binary_string, 2)


def eca_get_lut(rule_num: int) -> np.ndarray:
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(32)], dtype=np.uint8)


def eca_evolve(lut: np.ndarray, x: np.ndarray) -> np.ndarray:
    return lut[31 - ((np.roll(x, 2) * 16) + (np.roll(x, 1) * 8) + (x * 4) + (np.roll(x, -1) * 2) + np.roll(x, -2))]


def eca_evolve_spacetime(lut: np.ndarray, initial_conf: np.ndarray, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(eca_evolve(lut, rows[-1]))
    return np.stack(rows)


In [45]:
import numpy as np
from joblib import Parallel, delayed


def evaluate_lut(lut, configurations, steps):
    """Evaluate CA rule LUT on all given configurations, ensure no invalid fitness."""
    correct = 0
    for config in configurations:
        result = eca_evolve_spacetime(lut, config, steps)
        if DCP(result):
            correct += 1
    return correct / len(configurations)


# def fitness_on_population(population, configurations, steps):
#     """Compute fitness for each LUT in the population."""
#     return np.array([evaluate_lut(lut, configurations, steps) for lut in population])


def fitness_on_population(population, configurations, steps):
    """Compute fitness for each LUT in the population using joblib for parallel processing."""
    results = Parallel(n_jobs=-1)(delayed(evaluate_lut)(lut, configurations, steps) for lut in population)
    return np.array(results)


def choice(population, fitness_results):
    """Roulette wheel selection with safety for zero fitness sum."""
    total_fitness = np.sum(fitness_results)
    if total_fitness == 0:
        return np.random.choice(population)
    probabilities = fitness_results / total_fitness
    chosen_index = np.random.choice(len(population), p=probabilities)
    return population[chosen_index]


# 1
def cross(ind1, ind2):
    """Uniform crossover for binary LUTs."""
    mask = np.random.randint(2, size=len(ind1))
    child = np.where(mask, ind1, ind2)
    return child


# 2
# def cross(ind1, ind2):
#     """Single-point crossover for binary LUTs."""
#     crossover_point = np.random.randint(1, len(ind1))  # Choose a random crossover point
#     child = np.concatenate((ind1[:crossover_point], ind2[crossover_point:]))
#     return child

# 1
# def mutate(individual, mutation_rate=0.1):
#     """Bit flip mutation for LUTs."""
#     mutation_mask = np.random.rand(len(individual)) < mutation_rate
#     individual[mutation_mask] = 1 - individual[mutation_mask]
#     return individual

# 2
def mutate(individual, mutation_rate=0.1):
    """Bit flip mutation for LUTs with position-based mutation rate."""
    mutated_individual = individual.copy()
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate * (i / len(individual)):  # Adjust mutation rate based on position
            mutated_individual[i] = 1 - mutated_individual[i]  # Flip the bit
    return mutated_individual


def return_config(length):
    individual = np.zeros(gene_length, dtype=int)
    individual[np.random.randint(length)] = 1
    return individual


def run_evolution(configurations, steps, population_size, gene_length, num_iterations):
    #population = [np.random.randint(2, size=gene_length) for _ in range(population_size)]

    population = [return_config(gene_length) for _ in range(population_size)]  # Config with one "1"

    for i in range(num_iterations):
        print(f'Epoch {i}')
        fitness_results = fitness_on_population(population, configurations, steps)
        print(fitness_results)
        new_population = [mutate(
            cross(
                choice(population, fitness_results),
                choice(population, fitness_results)
            )
        ) for _ in range(population_size)]
        population = new_population

    return population


# Evolution with automatically selecting best rules 
# def run_evolution(configurations, steps, population_size, gene_length, num_iterations, elite_size):
#     population = [np.random.randint(2, size=gene_length) for _ in range(population_size)]
#     
#     for _ in range(num_iterations):
#         print(f'Epoch {_}')
#         fitness_results = fitness_on_population(population, configurations, steps)
#         
#         sorted_indices = np.argsort(fitness_results)[::-1]
#         sorted_population = [population[i] for i in sorted_indices]
#         elite_population = sorted_population[:elite_size]
#         
#         new_population = elite_population
#         
#         while len(new_population) < population_size:
#             parent1 = choice(sorted_population, fitness_results)
#             parent2 = choice(sorted_population, fitness_results)
#             child = mutate(cross(parent1, parent2))
#             new_population.append(child)
#         
#         population = new_population
#     
#     return population



In [46]:
marcin = configuration_reader("ALL_N23.bin", 23, True)[:5000]

configurations = marcin
steps = 46 
population_size = 10
gene_length = 32
num_iterations = 20
elite_size = 5

final_population = run_evolution(configurations, steps, population_size, gene_length, num_iterations)
print(final_population)

Epoch 0
[0.5 0.5 0.5 0.5 0.5 0.  0.  0.  0.5 0.5]
Epoch 1
[0.000e+00 5.000e-01 5.000e-01 0.000e+00 5.000e-01 0.000e+00 5.000e-01
 5.000e-01 1.676e-01 2.000e-04]
Epoch 2
[0.5    0.0534 0.5    0.5    0.5    0.5    0.5    0.001  0.1404 0.1562]
Epoch 3
[2.0e-04 0.0e+00 5.0e-01 5.0e-01 4.1e-02 5.0e-01 0.0e+00 5.0e-01 5.0e-01
 5.0e-01]
Epoch 4
[3.626e-01 5.000e-01 2.000e-04 5.000e-01 2.614e-01 8.060e-02 5.000e-01
 5.000e-01 0.000e+00 5.000e-01]
Epoch 5
[0.3308 0.3626 0.011  0.5    0.0556 0.     0.5    0.5    0.5    0.5   ]
Epoch 6
[0.5    0.5    0.     0.5    0.5    0.056  0.3294 0.5    0.5    0.5   ]
Epoch 7
[5.00e-01 5.00e-01 6.94e-02 5.00e-01 5.00e-01 5.00e-01 5.56e-02 5.00e-01
 5.00e-01 2.00e-04]
Epoch 8
[3.452e-01 7.560e-02 5.000e-01 3.630e-01 4.618e-01 2.000e-04 5.000e-01
 2.000e-04 3.718e-01 2.000e-04]
Epoch 9
[4.040e-02 5.000e-01 5.000e-01 1.854e-01 2.000e-04 4.556e-01 3.472e-01
 4.070e-01 4.380e-02 3.462e-01]
Epoch 10
[0.5    0.     0.5    0.     0.363  0.5    0.0344 0.5    0.5    0

In [27]:
# cross 2
configurations = configuration_reader("ALL_N23.bin", 23, True)[:5000]
final_rules = [binary_array_to_int(lut) for lut in final_population]

for rule in final_rules:
    results = []
    for config in configurations:
        results.append(DCP(eca_evolve_spacetime(eca_get_lut(rule), config, 46)))

    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')

Rule 3542390659: 0.02% of correct solutions
Rule 3551305410: 50.02% of correct solutions
Rule 3551385336: 50.02% of correct solutions
Rule 3543963122: 50.02% of correct solutions
Rule 3542390430: 0.04% of correct solutions
Rule 3551368903: 50.0% of correct solutions
Rule 3551385219: 50.0% of correct solutions
Rule 3542914763: 0.26% of correct solutions
Rule 3542930115: 32.4% of correct solutions
Rule 3549288158: 50.02% of correct solutions


In [30]:
# cross 1
configurations = configuration_reader("ALL_N23.bin", 23, True)[:5000]
final_rules = [binary_array_to_int(lut) for lut in final_population]

for rule in final_rules:
    results = []
    for config in configurations:
        results.append(DCP(eca_evolve_spacetime(eca_get_lut(rule), config, 46)))

    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')

Rule 4209162639: 50.0% of correct solutions
Rule 4209294235: 50.0% of correct solutions
Rule 4209433474: 50.02% of correct solutions
Rule 4209154975: 50.0% of correct solutions
Rule 4209417198: 50.02% of correct solutions
Rule 4209423248: 15.18% of correct solutions
Rule 4226201245: 38.98% of correct solutions
Rule 4209171871: 50.0% of correct solutions
Rule 4209957806: 50.02% of correct solutions
Rule 3942030483: 48.7% of correct solutions


In [38]:
# new with selecting best rules in each step
configurations = configuration_reader("ALL_N23.bin", 23, True)[:5000]
final_rules = [binary_array_to_int(lut) for lut in final_population]

for rule in final_rules:
    results = []
    for config in configurations:
        results.append(DCP(eca_evolve_spacetime(eca_get_lut(rule), config, 46)))

    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')

Rule 3548038796: 50.02% of correct solutions
Rule 3548039064: 50.02% of correct solutions
Rule 3548104649: 50.0% of correct solutions
Rule 3548039113: 50.0% of correct solutions
Rule 3548030863: 50.0% of correct solutions
Rule 3547975564: 4.6% of correct solutions
Rule 3555936140: 0.04% of correct solutions
Rule 3547899280: 47.84% of correct solutions
Rule 3514483613: 50.0% of correct solutions
Rule 3548040844: 43.36% of correct solutions


In [43]:
# big population
configurations = configuration_reader("ALL_N23.bin", 23, True)[:5000]
final_rules = [binary_array_to_int(lut) for lut in final_population]

for rule in final_rules:
    results = []
    for config in configurations:
        results.append(DCP(eca_evolve_spacetime(eca_get_lut(rule), config, 46)))

    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')

Rule 4276864278: 35.62% of correct solutions
Rule 4294479021: 50.0% of correct solutions
Rule 4293452430: 50.02% of correct solutions
Rule 4229911781: 0.02% of correct solutions
Rule 4277795860: 18.92% of correct solutions
Rule 4294367773: 23.98% of correct solutions
Rule 4273594438: 46.34% of correct solutions
Rule 4160672021: 0.02% of correct solutions
Rule 4289175722: 50.02% of correct solutions
Rule 4210683311: 50.0% of correct solutions
Rule 4288394443: 50.0% of correct solutions
Rule 4290478331: 50.0% of correct solutions
Rule 4025067132: 50.02% of correct solutions
Rule 4021787082: 50.02% of correct solutions
Rule 4292605403: 50.0% of correct solutions
Rule 3954825002: 17.42% of correct solutions
Rule 4004568983: 33.96% of correct solutions
Rule 4275568082: 50.02% of correct solutions
Rule 3885994971: 4.84% of correct solutions
Rule 4288125635: 50.0% of correct solutions
Rule 4227854090: 50.02% of correct solutions
Rule 4291800190: 50.02% of correct solutions
Rule 4282376244: 0.

In [47]:
# configs with one "1"
configurations = configuration_reader("ALL_N23.bin", 23, True)[:5000]
final_rules = [binary_array_to_int(lut) for lut in final_population]

for rule in final_rules:
    results = []
    for config in configurations:
        results.append(DCP(eca_evolve_spacetime(eca_get_lut(rule), config, 46)))

    print(f'Rule {rule}: {round(100 * sum(results) / len(results), 2)}% of correct solutions')

Rule 72618144: 40.8% of correct solutions
Rule 6067208: 17.96% of correct solutions
Rule 10270016: 11.5% of correct solutions
Rule 14422112: 32.02% of correct solutions
Rule 1331396: 0.02% of correct solutions
Rule 6067784: 50.0% of correct solutions
Rule 39064592: 6.46% of correct solutions
Rule 1316932: 2.42% of correct solutions
Rule 114578528: 13.36% of correct solutions
Rule 5510248: 48.74% of correct solutions


In [ ]:
w zbiorze konfiguracji na których testujemy bierzemy konfiguracje o gęstości<0.5
fitnes na sprawdzać czy reguła zastosowana T razy na konfiguracji daje same 0
czy reguła zastosowana T razy na konfiguracji odwrotnej daje same 1
i tylko gdy oba są true to fitness += 1